### Домашнее задание

Спарсить цитаты с сайта Quotes to Scrape - https://quotes.toscrape.com.
Необходимо извлечь: 
1) цитаты, 
2) авторов, 
3) список тегов.

Задание со звёздочкой: спарсить все цитаты на сайте, а не только с первой страницы.

При парсинге желательно использовать разные методы для получения необходимой информации.

In [25]:
import pandas as pd
import requests
import json

from bs4 import BeautifulSoup as bs

In [26]:
def _one_quote_parser_(one_quote):
    one_quote_info = {}
    
    if not one_quote.find('span', {'class':"text"}):
        one_quote_info['qote_text'] = None
    else: 
        one_quote_info['qote_text'] = one_quote.find('span', {'class':"text"}).text
    
    if not one_quote.find('small', {'class':"author"}):
        one_quote_info['author'] = None
    else:
        one_quote_info['author'] = one_quote.find('small', {'class':"author"}).text
    
    if not one_quote.find('small', {'class':"author"}).parent.select_one('a')['href']:
        one_quote_info['author_href'] = None
    else:
        one_quote_info['author_href'] = 'https://quotes.toscrape.com' + \
                                        one_quote.find('small', {'class':"author"}).parent.select_one('a')['href']
    
    if not one_quote.find_all('a', {'class': 'tag'}):
        one_quote_info['tags'] = None
    else:
        tag_list = []
        quote_tags = one_quote.find_all('a', {'class': 'tag'})
        for tag in quote_tags:
            tag_list.append(tag.text)
        one_quote_info['tags'] = tag_list
    return one_quote_info 

In [27]:
def quote_parser():
    page = 1
    url = f'https://quotes.toscrape.com/page/{page}'
    params = {} # Оставил на всякий случай, хотя не нашел какие параметры можно передать
    quots_info = []
    while True:
        req = requests.get(url, params = params)
        if req.status_code != requests.codes.ok: # проверяем на код 200
            raise Exception(f"http code == {r.status_code}")
        else:
            html = bs(req.content,'html.parser')
            quots = html.find_all('div', {'class':'quote'})
            for one_quote in quots:
                quots_info.append(_one_quote_parser_(one_quote))
            print('Page', page, 'was successfuly proceed')
            page+=1
            url = f'https://quotes.toscrape.com/page/{page}'
            if (html.select_one('li[class=next]')!= None):
                continue
            else:
                print('Parsing was fineshed successfuly')
                break
    return quots_info

In [29]:
df = pd.DataFrame(quote_parser())
df

Page 1 was successfuly proceed
Page 2 was successfuly proceed
Page 3 was successfuly proceed
Page 4 was successfuly proceed
Page 5 was successfuly proceed
Page 6 was successfuly proceed
Page 7 was successfuly proceed
Page 8 was successfuly proceed
Page 9 was successfuly proceed
Page 10 was successfuly proceed
Parsing was fineshed successfuly


,qote_text,author,author_href,tags
0,“The world as we have created it is a process ...,Albert Einstein,https://quotes.toscrape.com/author/Albert-Eins...,"[change, deep-thoughts, thinking, world]"
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,https://quotes.toscrape.com/author/J-K-Rowling,"[abilities, choices]"
2,“There are only two ways to live your life. On...,Albert Einstein,https://quotes.toscrape.com/author/Albert-Eins...,"[inspirational, life, live, miracle, miracles]"
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,https://quotes.toscrape.com/author/Jane-Austen,"[aliteracy, books, classic, humor]"
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,https://quotes.toscrape.com/author/Marilyn-Monroe,"[be-yourself, inspirational]"
...,...,...,...,...
95,“You never really understand a person until yo...,Harper Lee,https://quotes.toscrape.com/author/Harper-Lee,[better-life-empathy]
96,“You have to write the book that wants to be w...,Madeleine L'Engle,https://quotes.toscrape.com/author/Madeleine-L...,"[books, children, difficult, grown-ups, write,..."
97,“Never tell the truth to people who are not wo...,Mark Twain,https://quotes.toscrape.com/author/Mark-Twain,[truth]
98,"“A person's a person, no matter how small.”",Dr. Seuss,https://quotes.toscrape.com/author/Dr-Seuss,[inspirational]


In [30]:
jdf = df.to_json(orient='records')
json_data = json.loads(jdf)
with open(f'Lesson3.json', 'w') as f:
    json.dump(json_data, f)